In [2]:
# from FONN import FONN
import pandas as pd
import numpy as np
import time
import sklearn.datasets as skd

In [ ]:
class VectorPass:
    
    
    def __init__(self):
        self.__compiled = False
        self.__layerAdded = False
        self.__fitted = False
        self.__loss = None
        self.__optimizer = None
        self.__activations = [ ]
        self.__dropouts = [ ]
        self.__layerDims = [ ]
        self.__layerType = [ ]
        self.__costs = [ ]
        self.__params = { }
        self.__fOuts = { }
        self.__pOuts = { }
        self.__bOuts = { }
        self.__layer_types = "dense".split(" ")
        self.__activation_types = "relu sigmoid softmax".split(" ")
        self.__loss_types = "binary_crossentropy categorical_crossentropy sparse_categorical_crossentropy".split(" ")
        self.__optimizer_types = "adam gradient_descent sgd rmsprop".split(" ")
        
        
    class __LayerTypeError(Exception):
        def __init__(self):
            self.message = "layer type is unknown."
            super().__init__(self.message)
    class __ActivationError(Exception):
        def __init__(self):
            self.message = "activation is unknown."
            super().__init__(self.message)
    class __LayerDimError(Exception):
        def __init__(self):
            self.message = "layer dimension can only be a positive integer."
            super().__init__(self.message)
    class __NotFitError(Exception):
        def __init__(self):
            self.message = "fit the model before adding layers."
            super().__init__(self.message)
    class __NotNumpyArraysError(Exception):
        def __init__(self):
            self.message = "Either of X and Y is not a numpy array."
            super().__init__(self.message)
    class __LossError(Exception):
        def __init__(self):
            self.message = "loss is unknown"
            super().__init__(self.message)
    class __OptimizerError(Exception):
        def __init__(self):
            self.message = "optimizer is unknown."
            super().__init__(self.message)
    class __CategoricalLossError(Exception):
        def __init__(self):
            self.message = "use sparse categorical crossentropy for non one hot encoded arrays."
            super().__init__(self.message) 
    class __CompileError(Exception):
        def __init__(self):
            self.message = "add a layer before compiling."
            super().__init__(self.message)
    class __ModelNotCompiledError(Exception):
        def __init__(self):
            self.message = "compile the model before training."
            super().__init__(self.message)
            
            
    def add_layer(self, layer_type, layer_dim, activation,dropout=0):
        if(type(layer_type) != str or layer_type.lower() not in self.__layer_types):
            raise self.__LayerTypeError
        if(type(layer_dim) != int or layer_dim < 1):
            raise self.__LayerDimError
        if(type(activation) != str or activation.lower() not in self.__activation_types):
            raise self.__ActivationError
        lenDim = len(self.__layerDims)
        if(self.__fitted == False):
            raise self.__NotFitError
        self.__layerType.append(layer_type.lower())
        self.__layerDims.append(layer_dim)
        self.__activations.append(activation.lower())
        nL = layer_dim
        nLm1 = self.__layerDims[lenDim - 1]
        var = 1
        if(activation.lower() in "leaky_relu relu".split(" ")):
            var = np.sqrt(2 / nLm1)
        self.__params[f"W{lenDim}"] = np.random.randn(nL, nLm1) * var
        self.__params[f"b{lenDim}"] = np.zeros((nL,1))
        self.__params[f"VW{lenDim}"] = np.zeros((nL,nLm1))
        self.__params[f"Vb{lenDim}"] = np.zeros((nL,1))
        self.__params[f"SW{lenDim}"] = np.zeros((nL,nLm1))
        self.__params[f"Sb{lenDim}"] = np.zeros((nL,1))
        self.__dropouts.append(1 - dropout)
        self.__layerAdded = True
        print("Adding Layer...")
        time.sleep(1)
        print("Done.")
        
        
    def fit(self, X, Y):
        if(type(X) != np.ndarray or type(Y) != np.ndarray):
            raise self.__NotNumpyArraysError
        self.X = X
        self.Y = Y
        self.__layerType.append("input")
        self.__activations.append(None)
        self.__layerDims.append(X.shape[0])
        self.__dropouts.append(1)
        self.__fitted = True
        print("Fitting to X and Y...")
        time.sleep(2)
        print("Done.")
        
        
    def compile(self, loss, optimizer,lr=-1,beta1=0.9,beta2=0.99,ep=1e-8):
        if (self.__layerAdded == False):
            raise self.__CompileError
        if(type(loss) != str or loss.lower() not in self.__loss_types):
            raise self.__LossError
        if(type(optimizer) != str or optimizer.lower() not in self.__optimizer_types):
            raise self.__OptimizerError
        if(np.sum(self.Y==0) + np.sum(self.Y==1) != self.Y.shape[1] and loss == "categorical_crossentropy"):
            raise self.__CategoricalLossError
        if(loss == "sparse_categorical_crossentropy"):
            self.Y = self.__onehotEncoder(self.Y)
        self.__loss = loss.lower()
        self.__optimizer = optimizer.lower()
        if(lr == -1):
            if(optimizer.lower() == "rmsprop" or optimizer.lower() == "adam"):
                self.__lr = 0.001
            else:
                self.__lr = 0.05
        else:
            self.__lr = lr
        self.__beta1 = beta1
        self.__beta2 = beta2
        self.__ep = ep
        self.__compiled = True
        print("Compiling...")
        time.sleep(2)
        print("Done.")
            
            
    def __onehotEncoder(self,Y):
        item_index_dict = { }
        i = 0
        for item in np.unique(Y):
            item_index_dict[item] = i
            i += 1
        modY = np.zeros((np.unique(Y).size,Y.size))
        i = 0
        for item in Y.T:
            itemIndex = item_index_dict[item[0]]
            modY[itemIndex,i] = 1
            i += 1
        return modY
    
    
    def __G(self, Z, function):
        if(function == "relu"):
            return np.maximum(0,Z)
        elif(function == "sigmoid"):
            return 1 / (1 + np.exp(-Z))
        elif(function == "softmax"):
            return np.exp(Z) / np.sum(np.exp(Z),axis=0)
        
        
    def __dG(self, Z, function):
        if(function == "relu"):
            return np.where(Z<=0,0,1)
        elif(function == "sigmoid"):
            sig = self.__G(Z, "sigmoid")
            return sig * (1 - sig)
        elif(function == "softmax"):
            soft = self.__G(Z, "softmax")
            return soft * (1 - soft)
    
    
    def __denseforward__(self, A, W, b):
        return np.dot(W, A) + b
    
    
    def __feedforward__(self, X, params, fOuts, activations, dropouts):
        fOuts['A0'] = X
        lenDim = len(self.__layerDims)
        for l in range(1,lenDim):
            WL = params[f'W{l}']
            bL = params[f'b{l}']
            ALm1 = fOuts[f'A{l-1}']
            activation = activations[l]
            drop = dropouts[l]
            if(self.__layerType[l] == "dense"):
                ZL = self.__denseforward__(ALm1, WL, bL) 
            AL = self.__G(ZL, activation)
            if(drop != 1):
                DL = np.random.randn(AL.shape[0],AL.shape[1])
                DL = DL < drop
                AL = AL * DL
                AL = AL / drop
                fOuts[f'D{l}'] = DL
            fOuts[f'A{l}'] = AL
            fOuts[f'Z{l}'] = ZL        
            
    def __densebackward__(self,) 
    def __backpropag__(self,Y,params,fOuts,bOuts,activations,dropouts,loss):
        lenDim = len(self.__layerDims)
        m = Y.shape[1]
        AL = fOuts[f'A{lenDim-1}']
        if loss == "binary_crossentropy":
            dAL = (-1 / m) * (np.divide(Y, AL) - np.divide(1-Y,1-AL))
        if loss in ["sparse_categorical_crossentropy","categorical_crossentropy"]:
            dAL = (-1 / m) * (np.divide(Y, AL))
        for l in reversed(range(1,lenDim)):
            ZL = fOuts[f'Z{l}']
            ALm1 = fOuts[f'A{l-1}']
            WL = params[f'W{l}']
            drop = dropouts[l]
            activation = activations[l]
            dZL = dAL * self.__dG(ZL, activation)
            dWL = (1 / m) * np.dot(dZL,ALm1.T)
            dbL = (1 / m) * np.sum(dZL, axis = 1, keepdims = True)
            if(l!=1):
                dAL = np.dot(WL.T,dZL)
                if(drop != 1):
                    D = fOuts[f'D{l-1}']
                    dAL = dAL * D
                    dAL = dAL / drop
            bOuts[f'dW{l}'] = dWL
            bOuts[f'db{l}'] = dbL

            
    def __update_params__(self,fOuts,bOuts,params,optimizer):
        lenDims = len(self.__layerDims)
        if(optimizer == "gradient_descent"):
            for l in range(1,lenDims):
                W = params[f'W{l}']
                b = params[f'b{l}']
                dW = bOuts[f'dW{l}']
                db = bOuts[f'db{l}']
                W = W - self.__lr * dW
                b = b - self.__lr * db
                params[f'W{l}'] = W
                params[f'b{l}'] = b
        if(optimizer == "sgd"):
            for l in range(1,lenDims):
                W = params[f'W{l}']
                b = params[f'b{l}']
                dW = bOuts[f'dW{l}']
                db = bOuts[f'db{l}']
                VW = params[f'VW{l}']
                Vb = params[f'Vb{l}']
                VW = self.__beta1 * VW + (1 - self.__beta1) * dW
                Vb = self.__beta1 * Vb + (1 - self.__beta1) * db
                W = W - self.__lr * VW
                b = b - self.__lr * Vb
                params[f'W{l}'] = W
                params[f'b{l}'] = b
                params[f'VW{l}'] = VW
                params[f'VW{l}'] = Vb
        
    
    def __calculate_loss__(self, Y, fOuts):
        A = fOuts[f'A{len(self.__layerDims) - 1}']
        if(self.__loss == "binary_crossentropy"):
            return (-1 / Y.shape[1]) * np.sum(np.multiply(Y, np.log(A)) + np.multiply(1 - Y, np.log(1 - A)))
        if(self.__loss in ["sparse_categorical_crossentropy","categorical_crossentropy"]):
            return (-1 / Y.shape[1]) * np.sum(np.multiply(Y, np.log(A)))
        
    def train(self,epochs):
        print("Starting Training...")
        time.sleep(2)
        timeBTrain = time.time()
        if(self.__compiled == False):
            raise self.__ModelNotCompiledError
        for epoch in range(1,epochs+1):
            self.__feedforward__(self.X, self.__params, self.__fOuts, self.__activations, self.__dropouts)
            self.__backpropag__(self.Y, self.__params, self.__fOuts, self.__bOuts, self.__activations, self.__dropouts, self.__loss)
            self.__update_params__(self.__fOuts, self.__bOuts, self.__params, self.__optimizer)
            loss = self.__calculate_loss__(self.Y, self.__fOuts)
            self.__costs.append(loss)
            if(epoch % 100 == 0):
                print(f"Loss After epoch {epoch} is {loss}")
        timeATrain = time.time()
        print("-------------------------------------------------------------------------------------------------------------------------------")
        print("Training Complete...")
        time.sleep(1)
        print(f"Final Loss :- {self.__costs[-1]}.")
        print(f"Total Training Time :- {timeATrain - timeBTrain} secs")
            

In [440]:
 #TODO add sparse_categorical_entropy loss, add convolutions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
class FONN():
    
    #INITIALIZER
    
    def __init__(self, mini_batch_size = None, classes = None):
        #CONSTANTS
        self.__mini_batch_size = mini_batch_size
        self.__classes = classes
        self.__lossFunc = None
        self.__optimizer = None
        #LISTS
        self.__costs = [ ]
        self.__accuracy = [ ]
        self.__layer_dims = [ ]
        self.__dropouts = [ 1 ]
        self.__activations = [ "None" ]
        #DICTIONARIES
        self.__params = { }
        self.__nOuts = { }
        self.__pOuts = { }
        self.__pDerivs = { }
    #USER UNSUABLE FUNCTIONS
    
    #EXCEPTIONS REQUIRED FOR THE CLASS
    class NotNumpyArraysError(Exception):
        def __init__(self):
            self.message = "Either of X and Y is not a numpy array."
            super().__init__(self.message)
    class ActivationError(Exception):
        def __init__(self):
            self.message = "Activation is not defined."
            super().__init__(self.message)
    class ModelNotFitError(Exception):
        def __init__(self):
            self.message = "Please fit the model before adding a layer."
            super().__init__(self.message)
    class ModelNotCompiledError(Exception):
        def __init__(self):
            self.message = "Please compile the model before training."
            super().__init__(self.message)
    #MINI BATCH CREATOR
    def __create_mini_batches(self, X, Y): 
        minibatch = [ ]
        mbatch_size = self.__mini_batch_size
        m = X.shape[ 1 ]
        num_mini_batches = math.floor( m / mbatch_size)
        perms = np.random.permutation(m)
        shuff_X = X[ :, perms ]
        shudd_Y = Y[ :, perms ]
        for num in range(num_mini_batches):
            m_X = X[ :, num * mbatch_size: (num + 1) * mbatch_size ]
            m_Y = Y[ :, num * mbatch_size: (num + 1) * mbatch_size ]
            mbatch = ( m_X, m_Y )
            minibatch.append(mbatch)
        if(m % num_mini_batches != 0):
            m_X = X[ :, num_mini_batches * mbatch_size: ]
            m_Y = Y[ :, num_mini_batches * mbatch_size: ]
            mbatch = ( m_X, m_Y )
            minibatch.append(mbatch)
        return minibatch
    
    #ONE HOT ENCODER FOR Y
    def __runOneHot(self, Y): #TODO remove this altogether and make user use sparse categorical crossentropy
        classes = self.__classes
        modY = np.zeros(( classes, Y.shape[1] ))
        Y = Y.astype('object')
        for m in range(Y.shape[1]):
            modY[Y[0,m], m] = 1
        return modY
    
    #ACTIVATION FUNCTIONS AND THEIR DERIVATIVES
    def __G(self, z, activation):
        if(activation == "sigmoid"):
            return 1 / (1 + np.exp(-z))
        if(activation == "relu"):
            return np.maximum(0, z)
        if(activation == "softmax"):
            return (np.exp(z)) / (np.sum(np.exp(z), axis=0))
    def __GD(self, z, activation):
        if(activation == "sigmoid"):
            sig = self.__G(z, "sigmoid")
            return sig * (1 - sig)
        if(activation == "relu"):
            return np.where(z <= 0, 0, 1)
        if(activation == "softmax"):
            soft = self.__G(z, "softmax")
            return soft * (1 - soft)
        
    #FEED FORWARD FUNCTION
    def __fForward(self, X, params, nOuts, activations, dropout):
        l = len(self.__layer_dims)
        nOuts['A0'] = X
        m = X.shape[1]
        for i in range(1, l):
            W = params[f'W{i}']
            b = params[f'b{i}']
            A = nOuts[f'A{i-1}']
            activation = activations[i]
            Z = np.dot(W,A) + b
            A1 = self.__G(Z, activation)
            keep_prob = dropout[i]
            if(keep_prob!=1):
                D1 = np.random.rand(A1.shape[0], A1.shape[1])
                D1 = D1 < keep_prob
                A1 = A1 * D1
                A1 = A1 / keep_prob
                nOuts[f'D{i}'] = D1
            nOuts[f'Z{i}'] = Z
            nOuts[f'A{i}'] = A1
    
    #LOSS AND ACCURACY CALCULATOR
    def __calLoss(self, Y, nOuts, lossFunc): #TODO make this better
        l = len(self.__layer_dims)
        A = nOuts[f'A{l-1}']
        m = Y.shape[1]
        if(lossFunc == "binary_crossentropy"):
            loss = ((-1/m)*np.sum(np.multiply(Y,np.log(A)) + np.multiply(1-Y,np.log(1-A))))
            acc = ((np.sum(A.round()==Y)))*100
        if(lossFunc == "categorical_crossentropy"):
            acc=0
            for i in range(m):
                acc = acc + (A[:,i].round()==Y[:,i]).all()
            acc *=100
            loss = ((-1/m)*np.sum(np.multiply(Y,np.log(A))))
        return loss,acc
        
    #BACKPROPAGATION FUNCTION
    def __bPropagation(self, Y, nOuts, params, pDeriv, lossFunc, dropout): 
        l = len(self.__layer_dims)
        A = nOuts[f'A{l-1}']        
        m = Y.shape[1]
        dal = A - Y
        for i in reversed(range(1,l)):
            Z = nOuts[f'Z{i}']
            W = params[f'W{i}']
            b = params[f'b{i}']
            keep_prob = dropout[i-1]
            A_prev = nOuts[f'A{i-1}']
            activation = self.__activations[i]
            dz = dal * self.__GD(Z,activation)
            dW = (1/m)*np.dot(dz,A_prev.T)
            db = (1/m)*np.sum(dz,axis=1,keepdims=True)
            dal = np.dot(W.T,dz)
            if(keep_prob!=1):
                D = nOuts[f'D{i-1}']
                dal = dal * D
                dal = dal / dropout[i]
            pDeriv[f'dW{i}'] = dW
            pDeriv[f'db{i}'] = db
    
    #PARAMETER UPDATER FUNCTION
    def __updateParams(self,params,pDeriv,optimizer,lr,t,epsilon,beta1,beta2): #TODO make this cleaner
        l = len(self.__layer_dims)
        if(optimizer == "gradient_descent"):
            for i in range(1,l):
                params[f'W{i}'] = params[f'W{i}'] - lr * pDeriv[f'dW{i}']
                params[f'b{i}'] = params[f'b{i}'] - lr * pDeriv[f'db{i}'] 
        if(optimizer == "gradient_descent_momentum"):
            for i in range(1,l):
                params[f'VW{i}'] = beta1 * params[f'VW{i}'] + (1-beta1) * pDeriv[f'dW{i}']
                params[f'Vb{i}'] = beta1 * params[f'Vb{i}'] + (1-beta1) * pDeriv[f'db{i}']     
                params[f'W{i}'] = params[f'W{i}'] - lr * params[f'VW{i}']
                params[f'b{i}'] = params[f'b{i}'] - lr * params[f'Vb{i}']
        if(optimizer == "RMSProp"):
            for i in range(1,l):
                dW = pDeriv[f'dW{i}']
                db = pDeriv[f'db{i}']
                SW = params[f'SW{i}']
                Sb = params[f'Sb{i}']
                
                SW = beta1 * SW + (1 - beta1) * dW * dW
                Sb = beta1 * Sb + (1 - beta1) * db * db
                
                params[f'SW{i}'] = SW/(1-beta1**t)
                params[f'Sb{i}'] = Sb/(1-beta1**t)
                
                params[f"W{i}"] -= lr * dW/np.sqrt(SW + epsilon)
                params[f"b{i}"] -= lr * db/np.sqrt(Sb + epsilon)
        if(optimizer == "Adam"):
            for i in range(1,l):
                dW = pDeriv[f'dW{i}']
                db = pDeriv[f'db{i}']
                SW = params[f'SW{i}']
                Sb = params[f'Sb{i}']
                VW = params[f'SW{i}']
                Vb = params[f'Sb{i}']
                
                SW = beta1 * SW + (1 - beta1) * dW * dW
                Sb = beta1 * Sb + (1 - beta1) * db * db
                
                params[f'SW{i}'] = SW/(1-beta1**t)
                params[f'Sb{i}'] = Sb/(1-beta1**t)
                
                params[f'VW{i}'] = beta1 * params[f'VW{i}'] + (1-beta1) * pDeriv[f'dW{i}']
                params[f'Vb{i}'] = beta1 * params[f'Vb{i}'] + (1-beta1) * pDeriv[f'db{i}']     
                params[f'W{i}'] = params[f'W{i}'] - lr * params[f'VW{i}']/np.sqrt(SW + epsilon)
                params[f'b{i}'] = params[f'b{i}'] - lr * params[f'Vb{i}']/np.sqrt(Sb + epsilon)
                
            
        
    #USER USABLE FUNCTIONS
    
    #FITTING THE MODEL TO INPUTS AND OUTPUTS
    def fit(self,X,Y,onehot): 
        
        if(type(X) != np.ndarray or type(Y) != np.ndarray):
            raise self.__NotNumpyArraysError
        if(onehot==True):
            Y = self.__runOneHot(Y)
        if(self.__mini_batch_size==None):
            batch = [(X,Y)]
        else:
            batch = self.__create_mini_batches(X,Y) 
        self.__batch = batch
        self.__layer_dims.append(X.shape[0])
        self.__X = X
        self.__Y = Y
    
    #COMPILING THE MODEL
    def compile(self,lossFunc,optimizer,lr=0.03,beta1=0.9,beta2=0.999,epsilon=1e-8): #TODO add exceptions for unknown loss and
                                                                                     #and optimizers
        self.__lossFunc = lossFunc
        self.__optimizer = optimizer
        self.__lr = lr
        self.__beta1 = beta1
        self.__beta2 = beta2
        self.__epsilon = epsilon
    #ADDING LAYERS TO MODEL    
    def addLayer(self,layer_dim,activator,dropout=0,layer_type=None): #TODO separate dropout as a layer
        if(activator not in "sigmoid relu softmax".split(" ")):
            raise self.__ActivationError
        else:
            if(len(self.__layer_dims)==0):
                raise self.__ModelNotFitError
            else:
                l = len(self.__layer_dims)
                self.__layer_dims.append(layer_dim)
                self.__activations.append(activator)
                self.__dropouts.append(1-dropout)
                nH = layer_dim
                nHm1 = self.__layer_dims[l-1]
                self.__params[f'W{l}'] = np.random.randn(nH,nHm1)*np.sqrt(2/nHm1)
                self.__params[f'b{l}'] = np.zeros((nH,1))
                self.__params[f'SW{l}'] = np.zeros(self.__params[f'W{l}'].shape)
                self.__params[f'Sb{l}'] = np.zeros(self.__params[f'b{l}'].shape)
                self.__params[f'VW{l}'] = np.zeros(self.__params[f'W{l}'].shape)
                self.__params[f'Vb{l}'] = np.zeros(self.__params[f'b{l}'].shape)
                
    #TRAINING THE MODEL
    def train(self,epochs,getCost=False): #TODO remove get cost and always return a cache containing all data
        if(self.__lossFunc == None): 
            raise self.__ModelNotCompiledError
        pDeriv = self.__pDerivs
        params = self.__params
        nOuts = self.__nOuts
        epsilon = self.__epsilon
        beta1 = self.__beta1
        beta2 = self.__beta2 
        getCost = getCost
        optimizer = self.__optimizer
        activations = self.__activations
        dropouts = self.__dropouts
        lr = self.__lr
        lossFunc = self.__lossFunc
        t = 0
        for epoch in range(epochs+1): #TODO make this code a bit better
            costtotal = 0
            acctotal = 0
            for batch in self.__batch:
                X = batch[0]
                Y = batch[1]
                self.__fForward(X,params,nOuts,activations,dropouts)
                loss,acc = self.__calLoss(Y,nOuts,lossFunc)
                costtotal = costtotal + loss
                acctotal = acctotal + acc
                self.__bPropagation(Y,nOuts,params,pDeriv,lossFunc,dropouts)
                t += 1
                self.__updateParams(params,pDeriv,optimizer,lr,t,epsilon,beta1,beta2)
            self.__costs.append(costtotal/self.__X.shape[1])
            self.__accuracy.append(acctotal/self.__X.shape[1])
            if(epoch%10 == 0):
                print(f"Loss after epoch {epoch} is {self.__costs[-1]} and accuracy is {self.__accuracy[-1]}.")
        if(getCost):
            return self.__costs,self.__accuracy
    
    #PREDICTING FOR TEST SET/DEV SET
    def predict(self,X,Y,onehot=False): #TODO separate predict and evaluate functions
        l = len(self.__layer_dims)
        if(onehot):
            Y = self.__runOneHot(Y)
        pOuts = self.__pOuts
        params = self.__params
        dropouts = self.__dropouts
        activations = self.__activations
        self.__fForward(X,params,pOuts,activations,dropouts)
        A = pOuts[f'A{l-1}']
        if(self.__lossFunc == "categorical_crossentropy"):
            acc = 0
            for i in range(Y.shape[1]):
                acc += (A[:,i].round() == Y[:,i]).all() 
            acc /= Y.shape[1]
        else:
            acc = ((np.sum(A.round()==Y)))/Y.shape[1]
        print(f"Accuracy on the test set is {acc*100}.")

    #PLOTS TWO SEPARATE GRAPHS FOR ACCURACY AND COSTW
    def plotCostAndAccuracy(self): #TODO add a bool if a user wants to plot after training directly
        plt.plot(self.__costs)
        plt.xlabel("EPOCHS")
        plt.ylabel("COST")
        plt.title("COST vs EPOCH GRAPH")
        plt.show()
        plt.plot(self.__accuracy)
        plt.xlabel("EPOCHS")
        plt.ylabel("ACCURACY")
        plt.title("ACCURACY vs EPOCH GRAPH")
        plt.show()
#© Aditya Rangarajan 2020

In [441]:
X = skd.load_digits()['data'].T
Y = skd.load_digits()['target'].reshape(-1,1).T
X = X/255.0

In [446]:
m = FONN(classes = 10)

In [447]:
Y


array([[0, 1, 2, ..., 8, 9, 8]])

In [448]:
m.fit(X,Y,True)

In [427]:
m.add_layer("dense",128,"relu")
m.add_layer("dense",128,"relu")
m.add_layer("dense",64,"relu")
m.add_layer("dense",10,"sigmoid")

Adding Layer...
Done.
Adding Layer...
Done.
Adding Layer...
Done.
Adding Layer...
Done.


In [428]:
m.compile("sparse_categorical_crossentropy","gradient_descent")

Compiling...
Done.


In [429]:
m.train(1000)

Starting Training...
Loss After epoch 100 is 0.6927081103118738
Loss After epoch 200 is 0.6926344085193705
Loss After epoch 300 is 0.692560716496555
Loss After epoch 400 is 0.6924870341178491
Loss After epoch 500 is 0.692413361414116
Loss After epoch 600 is 0.6923396982627772
Loss After epoch 700 is 0.6922660447128837
Loss After epoch 800 is 0.6921924008181758
Loss After epoch 900 is 0.6921187664525327
Loss After epoch 1000 is 0.6920451416258101
-------------------------------------------------------------------------------------------------------------------------------
Training Complete...
Final Loss :- 0.6920451416258101.
Total Training Time :- 19.023137092590332 secs


In [435]:
A = m._VectorPass__fOuts['A4']
Y = m._VectorPass__onehotEncoder(Y)

In [436]:
acc=0
for i in range(Y.shape[1]):
    acc = acc + (A[:,i].round()==Y[:,i]).all()


In [439]:
A.round()

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [0., 1., 0., ..., 0., 1., 0.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [430]:
m._VectorPass__lr

0.05

In [4]:
np.maximum(0,0)

0